In [ ]:
from sklearn import datasets

iris = datasets.fetch_covtype()

In [ ]:
data = iris['data'][:5000]

In [ ]:
target = iris["target"][:5000]

In [ ]:
data.shape

(1000, 54)

In [ ]:
import numpy as np

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.33, random_state=42)

In [ ]:
y_train.shape

(3350,)

In [ ]:
import numpy as np
from collections import Counter
from math import inf
class Leaf():
  def __init__(self, y):
      self.y = y

class Node():
  def __init__(self, left, right, condition):
      self.left = left
      self.right = right
      self.condition = condition

class DecisionTree():
  def __init__(self,max_depth):
      self.max_depth = max_depth

  def impurity(self, y):
      h_q = 0
      cnt = Counter(y)
      for key in cnt:
        cnt[key]/=len(y)

      for key in cnt:
        h_q+=cnt[key]*(1-cnt[key])

      return len(y)*h_q


  def split(self,x ,y):
    min_impurity = inf
    best_idx_split = -1
    best_condition = None
    best_x_left = None
    best_x_right = None
    best_y_left = None
    best_y_right = None
    # best_
    for col in tqdm(range(x.shape[1])):
        idxs  = np.argsort(x[:,col])
        new_y = y[idxs]

        for split_idx in range(1,len(x)):
          new_y_left = new_y[:split_idx]
          new_y_right = new_y[split_idx:]

          impurity_y_1 = self.impurity(new_y_left)
          impurity_y_2 = self.impurity(new_y_right)
          sum_of_impurity = impurity_y_1 + impurity_y_2

          if sum_of_impurity < min_impurity:
            best_x_left = x[idxs[:split_idx], :]
            best_x_right = x[idxs[split_idx:], :]
            min_impurity = sum_of_impurity
            best_y_left = new_y_left
            best_y_right = new_y_right
            best_condition = (col, x[idxs][split_idx, col])
    # print()
    return best_x_left,best_y_left,best_x_right,best_y_right, best_condition

  def build_tree(self, x, y, depth):
    node = Node(None,None,None)

    if depth == self.max_depth:
      if len(y)!=0:
        cnt = Counter(y)
        y = cnt.most_common(1)[0][0]
      return Leaf(y)

    x1, y1, x2, y2, condition = self.split(x,y)
    if x1 is None:
      cnt = Counter(y)
      y = cnt.most_common(1)[0][0]
      return Leaf(y)
    node.condition = condition
    node.left = self.build_tree(x1, y1,depth+1)
    node.right = self.build_tree(x2, y2,depth+1)
    return node


In [ ]:
def inference(node,x):
  if isinstance(node,Leaf):
    return node.y
  if x[node.condition[0]] < node.condition[1]:
    return inference(node.left,x)
  else:
    return inference(node.right,x)

In [ ]:
tree = DecisionTree(3)
dec_tree = tree.build_tree(X_train[:200],y_train[:200],0)

100%|██████████| 54/54 [00:00<00:00, 439.76it/s]


In [156]:
from tqdm import tqdm
right = 0
for i in tqdm(range(len(X_test))):
  cls = inference(dec_tree,X_test[i])
  if cls == y_test[i]:
    right+=1
print(right/len(y_test))

100%|██████████| 1650/1650 [00:00<00:00, 365299.64it/s]

0.6242424242424243


In [ ]:
N = 10
trees = []
subsample_n = 1000
for i in tqdm(range(N)):
  tree = DecisionTree(3)
  idxs = np.random.choice(len(X_train),subsample_n)
  dec_tree = tree.build_tree(X_train[idxs],y_train[idxs],0)
  trees.append(dec_tree)

In [ ]:
def inference_trees(trees,X_test):
  all_x_ans = []
  for i in tqdm(range(len(X_test))):
    ans = []
    for j in range(len(trees)):
      cls = inference(trees[j],X_test[i])
      ans.append(cls)
    cnt = Counter(ans)
    all_x_ans.append(cnt.most_common(1)[0][0])
  return all_x_ans

In [ ]:
ans = inference_trees(trees,X_test)

100%|██████████| 1650/1650 [00:00<00:00, 42750.11it/s]


In [ ]:
ans[:50]

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
accuracy_score(ans,y_test)

0.6309090909090909